# Explore dataset and transformations

### Import necessary stuff, define helper functions and take some sample images

In [ ]:
import tensorflow as tf

from dataset import ds, apply_transformations
from dataset.transformations import drop_alpha_channel
from dataset.utils import new_summary_writer, draw_label

N_SAMPLES = 128

To avoid repeating code, let's define a little helper to draw label marks and write images to TensorBoard.

In [2]:
def write_previews(name, preview_dataset):
    with_labels = [draw_label(x, y, block_size=15) for x, y in preview_dataset]

    with new_summary_writer().as_default():
        tf.summary.image(name, with_labels,
                         max_outputs=len(with_labels), step=0)

Shuffle whole untouched dataset and take (preload) `N_SAMPLES` for previews.

In [3]:
samples = [(x, y) for x, y in ds.shuffle(ds.cardinality()).take(N_SAMPLES)]
preview_ds = tf.data.Dataset.from_tensor_slices((
    [x for x, _ in samples],
    [y for _, y in samples]
    ))

Again, to avoid repeating code, we define helper to apply transforamtions to preview dataset and then write results in TensorBoard.

In [4]:
f = {'factor': 0.5}

def apply_and_write(name, transformations):
    transformed = apply_transformations(preview_ds,
                                        [drop_alpha_channel, *transformations],
                                        batch=N_SAMPLES)
    write_previews(name, transformed)

### Write untouched images to TensorBoard

In [5]:
write_previews('Clean dataset', preview_ds)

### Apply transformations one by one and write results into TensorBoard

In [6]:
from dataset.transformations import flip_lr, flip_ud

apply_and_write('Random flips', [flip_lr(**f), flip_ud(**f)])

In [7]:
from dataset.transformations import inverse

apply_and_write('Random inverse', [inverse(**f)])

In [8]:
from dataset.transformations import remove_random_channel

apply_and_write('Random channel remove', [remove_random_channel(**f)])

In [9]:
from dataset.transformations import random_brightness

apply_and_write('Random brightness', [random_brightness(**f)])

In [10]:
from dataset.transformations import random_contrast

apply_and_write('Random contrast', [random_contrast(**f)])

In [11]:
from dataset.transformations import random_hue

apply_and_write('Random HUE', [random_hue(**f)])

In [12]:
from dataset.transformations import random_saturation

apply_and_write('Random saturation', [random_saturation(**f)])

In [13]:
from dataset.transformations import noise

apply_and_write('Random noise', [noise(**f)])

In [14]:
from dataset.transformations import new_depthwise_gaussian_kernel, gaussian_blur

kernel = new_depthwise_gaussian_kernel(5, 0, 2)
apply_and_write('Random Gaussian blur', [gaussian_blur(kernel=kernel, **f)])

In [15]:
from dataset.transformations import add_grid_to_image

apply_and_write('Random grid', [add_grid_to_image(**f)])

### Apply all the transformations at once and see what happens

In [16]:
from dataset.transformations import default_transformations

apply_and_write('Combined random transformations', default_transformations)